In [2]:
import time
import random
import csv
import os
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Fonction de pause aléatoire pour simuler un comportement humain
def human_pause(min_sec=2, max_sec=5):
    time.sleep(random.uniform(min_sec, max_sec))

# Initialisation des fichiers de log
log_date = datetime.now().strftime("%Y-%m-%d")
log_filename = f"log_consultations_{log_date}.csv"
error_log_filename = f"error_log_{log_date}.log"

with open(log_filename, mode='w', newline='', encoding='utf-8') as log_file:
    writer = csv.writer(log_file)
    writer.writerow(["Date", "URL Consultation", "Titre", "Acheteur", "Statut", "Erreur"])

# Fonction pour enregistrer les erreurs dans un fichier .log
def log_error(message):
    with open(error_log_filename, mode='a', encoding='utf-8') as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {message}\n")
    print(f"ERREUR: {message}")

# Fonction pour enregistrer les activités
def log_activity(url, titre="", acheteur="", statut="", erreur=""):
    with open(log_filename, mode='a', newline='', encoding='utf-8') as log_file:
        writer = csv.writer(log_file)
        writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), url, titre, acheteur, statut, erreur])

# Fonction pour créer un rapport détaillé des téléchargements
def create_download_report():
    try:
        report_filename = f"rapport_telechargements_{log_date}.txt"
        with open(report_filename, 'w', encoding='utf-8') as f:
            f.write("=" * 60 + "\n")
            f.write("RAPPORT DE TÉLÉCHARGEMENT - MARCHÉS PUBLICS MAROC\n")
            f.write("=" * 60 + "\n")
            f.write(f"Date d'exécution : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Répertoire de téléchargement : {download_dir}\n")
            f.write("\n")
            
            if os.path.exists(download_dir):
                files = os.listdir(download_dir)
                f.write(f"Nombre de fichiers téléchargés : {len(files)}\n")
                f.write("\nListe des fichiers :\n")
                for file_name in files: # Renamed 'file' to 'file_name' to avoid conflict
                    file_path = os.path.join(download_dir, file_name)
                    if os.path.isfile(file_path):
                        file_size = os.path.getsize(file_path)
                        f.write(f"- {file_name} ({file_size} octets)\n")
            else:
                f.write("Aucun fichier téléchargé\n")
                
        print(f"📄 Rapport créé : {report_filename}")
        return report_filename
    except Exception as e:
        print(f"Erreur création rapport : {e}")
        return None

# Configuration furtive de Selenium
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36")

# Configuration du répertoire de téléchargement
download_dir = os.path.join(os.getcwd(), "downloads")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safeBrowse.enabled": True
}
options.add_experimental_option("prefs", prefs)

# Initialisation du navigateur
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

print("🚀 Démarrage du scraper de marchés publics")

# Accès au site via Google
try:
    print("🔍 Recherche du site via Google...")
    driver.get("https://www.google.com")
    human_pause()
    
    try:
        # Essayer d'accepter les cookies sur Google
        accept_buttons = driver.find_elements(By.XPATH, '//button[div[contains(text(), "Accept all")]] | //button[div[contains(text(), "Tout accepter")]] | //button[contains(text(), "J\'accepte")] | //button[contains(text(), "Accept all")]')
        if accept_buttons:
            accept_buttons[0].click()
            print("Cookies Google acceptés.")
            human_pause()
    except Exception as e:
        print(f"Pas de bouton cookies Google ou erreur lors de l'acceptation : {e}")

    search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "q")))
    search_box.send_keys("marchés publics maroc")
    human_pause(1, 2)
    search_box.send_keys(Keys.RETURN)
    human_pause(3, 5)

    link = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'marchespublics.gov.ma')][h3]"))
    )
    print(f"🔗 Lien trouvé: {link.get_attribute('href')}")
    link.click()
    human_pause(5, 7)
    print("✅ Accès au site marchespublics.gov.ma réussi")
    
    try:
        WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'iframeconsent')))
        accept_button_site = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'axeptio_btn_acceptAll')))
        accept_button_site.click()
        driver.switch_to.default_content()
        print("🍪 Cookies du site des marchés publics acceptés.")
        human_pause(2,3)
    except TimeoutException:
        print("Pas de pop-up/cookies géré sur le site des marchés publics ou déjà accepté.")
        driver.switch_to.default_content()
    except Exception as e:
        print(f"Erreur lors de la gestion des cookies/pop-ups du site : {e}")
        driver.switch_to.default_content()

except Exception as e:
    log_error(f"Erreur accès site marchés publics : {e}")
    driver.quit()
    exit()

# Accès à la recherche avancée de consultations en cours
try:
    print("📋 Accès aux consultations en cours...")
    consultations_link = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Consultations en cours")))
    consultations_link.click()
    human_pause(5, 6)
    print("✅ Page consultations en cours chargée")
except Exception as e:
    log_error(f"Erreur lien 'Consultations en cours' : {e}")
    driver.quit()
    exit()

# Sélection catégorie "Services"
try:
    print("🏷️ Sélection de la catégorie Services...")
    select_cat_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_categorie"))
    )
    Select(select_cat_element).select_by_visible_text("Services")
    print("✅ Catégorie 'Services' sélectionnée.")
    human_pause(2, 3)
except Exception as e:
    log_error(f"Erreur sélection catégorie : {e}")

# Configuration des dates
print("📅 Configuration des dates...")

aujourd_hui = datetime.now()
date_2mois_plus_tard = (aujourd_hui + timedelta(days=60)).strftime('%d/%m/%Y') # Au moins 2 mois
delai_maximum = (aujourd_hui + timedelta(days=210)).strftime('%d/%m/%Y') # Au moins 9 mois
hier = (aujourd_hui - timedelta(days=1)).strftime('%d/%m/%Y')
aujourd_hui_str = aujourd_hui.strftime('%d/%m/%Y')

try:
    # Mise en ligne : d'hier à aujourd'hui
    champ_debut_pub = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_dateMiseEnLigneCalculeStart"))
    )
    champ_debut_pub.clear()
    champ_debut_pub.send_keys(hier)

    champ_fin_pub = driver.find_element(By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_dateMiseEnLigneCalculeEnd")
    champ_fin_pub.clear()
    champ_fin_pub.send_keys(aujourd_hui_str)

    # Remise des plis : d'aujourd'hui à aujourd'hui + 2 mois
    champ_debut_plis = driver.find_element(By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_dateRemisePlisStart")
    champ_debut_plis.clear()
    champ_debut_plis.send_keys(aujourd_hui_str)

    champ_fin_plis = driver.find_element(By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_dateRemisePlisEnd")
    champ_fin_plis.clear()
    champ_fin_plis.send_keys(date_2mois_plus_tard)
    
    print(f"✅ Dates configurées : Publication {hier}-{aujourd_hui_str}, Remise des plis {date_2mois_plus_tard}-{delai_maximum}")
    human_pause(1, 2)
    
except Exception as e:
    log_error(f"Erreur dans les champs de date : {e}")

# Lancer la recherche
try:
    print("🚀 Lancement de la recherche...")
    search_button = driver.find_element(By.ID, "ctl0_CONTENU_PAGE_AdvancedSearch_lancerRecherche")
    driver.execute_script("arguments[0].click();", search_button)
    print("✅ Recherche lancée via JavaScript")
    
    wait_conditions = [
        EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_resultSearch_listePageSizeTop")),
        EC.presence_of_element_located((By.CLASS_NAME, "table-results")),
        EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_resultSearch_panelElementsFound"))
    ]
    
    element_found = False
    for condition in wait_conditions:
        try:
            WebDriverWait(driver, 15).until(condition)
            print("✅ Page de résultats chargée")
            element_found = True
            break
        except TimeoutException:
            continue
    
    if not element_found:
        raise TimeoutException("Aucun élément de résultat de recherche (table, pagination) trouvé après clic sur 'Lancer la recherche'")
        
    human_pause(3, 5)
    
except Exception as e:
    log_error(f"Erreur lancement recherche : {e}")
    print("⚠️ Tentative de continuation malgré l'erreur de chargement des résultats...")

# Configuration de la pagination
try:
    print("📄 Configuration de la pagination...")
    pagination_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_resultSearch_listePageSizeTop"))
    )
    select_pagination = Select(pagination_element)
    
    # Choisir la valeur la plus élevée (500 si disponible, sinon la dernière option)
    highest_value_option = select_pagination.options[-1].get_attribute('value')
    if "500" in [opt.get_attribute('value') for opt in select_pagination.options]:
        select_pagination.select_by_value("500")
        print("✅ Pagination configurée à 500 résultats par page")
    else:
        select_pagination.select_by_value(highest_value_option)
        print(f"✅ Pagination configurée à {highest_value_option} résultats par page (valeur la plus élevée disponible)")

    human_pause(5, 7) # Attendre le rechargement
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "table-results")))
    print("✅ Page rechargée avec nouvelle pagination")
    
except Exception as e:
    log_error(f"Erreur configuration pagination : {e}")
    print("⚠️ Continuation avec pagination par défaut")

# Parcours des consultations et téléchargement
try:
    print("📊 Extraction des consultations...")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table_results = soup.find("table", class_="table-results")
    
    if not table_results:
        log_error("Tableau des résultats non trouvé après configuration pagination.")
        print("❌ Aucun tableau de résultats trouvé pour l'extraction.")
    else:
        rows = table_results.find("tbody").find_all("tr") if table_results.find("tbody") else table_results.find_all("tr")
        print(f"📋 {len(rows)} lignes trouvées dans le tableau")
        
        consultations_traitees = 0
        consultations_telechargees = 0
        
        for i, row in enumerate(rows):
            status = "Échec"
            error_msg = ""
            titre_consultation = "Titre non disponible"
            current_consultation_url_for_log = "URL non accessible"

            try:
                actions_cell = row.find("td", class_="actions")
                if not actions_cell:
                    continue
                
                consultation_link_tag = actions_cell.find("a", href=True, title="Accéder à la consultation")
                if not consultation_link_tag: # Fallback si title n'est pas là
                    consultation_link_tag = actions_cell.find("a", href=True) 
                if not consultation_link_tag:
                    continue
                
                href = consultation_link_tag.get('href')
                if not href or href == "":
                    continue
                
                if href.startswith('/'):
                    consultation_url = "https://www.marchespublics.gov.ma" + href
                elif not href.startswith('http'):
                    consultation_url = "https://www.marchespublics.gov.ma/" + href # Assumer que c'est relatif au domaine
                else:
                    consultation_url = href
                
                current_consultation_url_for_log = consultation_url
                consultations_traitees += 1
                print(f"\n📋 [{consultations_traitees}/{len(rows)}] Traitement consultation: {consultation_url}")
                
                driver.get(consultation_url)
                human_pause(3, 5)
                
                try:
                    titre_consultation = driver.title or "Titre non disponible"
                except:
                    pass # titre_consultation reste "Titre non disponible"

                # --- PROCESSUS DE TÉLÉCHARGEMENT EN PLUSIEURS ÉTAPES ---
                try:
                    print("🔗 Recherche du lien 'Dossier de consultation' (vers formulaire)...")
                    dossier_consultation_link_to_form = WebDriverWait(driver, 15).until(
                        EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_linkDownloadDce")) #
                    )
                    print("📄 Lien 'Dossier de consultation' (vers formulaire) trouvé.")
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", dossier_consultation_link_to_form)
                    human_pause(1,2)
                    dossier_consultation_link_to_form.click()
                    print("✅ Clic sur 'Dossier de consultation', navigation vers formulaire.")
                    human_pause(4, 6)

                    print("📝 Remplissage du formulaire de téléchargement...")

                    # Attendre que la page soit complètement chargée
                    time.sleep(3)

                    # Attendre que le formulaire principal soit présent
                    try:
                        WebDriverWait(driver, 20).until(
                            EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_panelEntrepriseFormulaireDemande"))
                        )
                        print("✅ Formulaire détecté")
                    except Exception as e:
                        print(f"❌ Formulaire non trouvé : {e}")
                        raise

                    # Attendre spécifiquement que le bloc des coordonnées soit présent
                    try:
                        WebDriverWait(driver, 15).until(
                            EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_EntrepriseFormulaireDemande__blocMesCoordonnees"))
                        )
                        print("✅ Bloc coordonnées détecté")
                    except Exception as e:
                        print(f"❌ Bloc coordonnées non trouvé : {e}")

                    # Données à remplir
                    form_data = {
                        "nom": "YONLI",
                        "prenom": "Fidèle", 
                        "email": "Fidele@immersion.ma"
                    }

                    # 1. Remplir le champ NOM
                    try:
                        print("🔍 Recherche du champ NOM...")
                        nom_field = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_EntrepriseFormulaireDemande_nom"))
                        )
    
                        # Scroll et focus sur l'élément
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", nom_field)
                        driver.execute_script("arguments[0].focus();", nom_field)
                        time.sleep(1)
    
                        # Effacer et remplir
                        nom_field.clear()
                        time.sleep(0.5)
                        nom_field.send_keys(form_data["nom"])
                        print(f"✅ Champ NOM rempli avec : {form_data['nom']}")
    
                    except Exception as e:
                        print(f"❌ Erreur champ NOM : {e}")

                    # 2. Remplir le champ PRÉNOM
                    try:
                        print("🔍 Recherche du champ PRÉNOM...")
                        prenom_field = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_EntrepriseFormulaireDemande_prenom"))
                        )
    
                        # Scroll et focus sur l'élément
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", prenom_field)
                        driver.execute_script("arguments[0].focus();", prenom_field)
                        time.sleep(1)
    
                        # Effacer et remplir
                        prenom_field.clear()
                        time.sleep(0.5)
                        prenom_field.send_keys(form_data["prenom"])
                        print(f"✅ Champ PRÉNOM rempli avec : {form_data['prenom']}")
    
                    except Exception as e:
                        print(f"❌ Erreur champ PRÉNOM : {e}")
                        # Essayer avec un sélecteur CSS alternatif
                        try:
                            print("🔍 Tentative alternative pour PRÉNOM...")
                            prenom_field_alt = driver.find_element(By.CSS_SELECTOR, "input[id*='prenomValidator']")
                            driver.execute_script("arguments[0].scrollIntoView(true);", prenom_field_alt)
                            driver.execute_script("arguments[0].focus();", prenom_field_alt)
                            time.sleep(1)
                            prenom_field_alt.clear()
                            prenom_field_alt.send_keys(form_data["prenom"])
                            print(f"✅ Champ PRÉNOM rempli (méthode alternative) avec : {form_data['prenom']}")
                        except Exception as e2:
                            print(f"❌ Erreur alternative PRÉNOM : {e2}")

                    # 3. Remplir le champ EMAIL
                    try:
                        print("🔍 Recherche du champ EMAIL...")
                        email_field = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_EntrepriseFormulaireDemande_email"))
                        )
    
                        # Scroll et focus sur l'élément
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", email_field)
                        driver.execute_script("arguments[0].focus();", email_field)
                        time.sleep(1)
    
                        # Effacer et remplir
                        email_field.clear()
                        time.sleep(0.5)
                        email_field.send_keys(form_data["email"])
                        print(f"✅ Champ EMAIL rempli avec : {form_data['email']}")
    
                    except Exception as e:
                        print(f"❌ Erreur champ EMAIL : {e}")

                    # 4. Cocher la case des conditions générales
                    try:
                        print("🔍 Recherche de la case à cocher...")
                        checkbox = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_EntrepriseFormulaireDemande_accepterConditions"))
                        )
    
                        # Scroll vers la checkbox
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", checkbox)
                        time.sleep(1)
    
                        # Vérifier si déjà cochée
                        if not checkbox.is_selected():
                            # Essayer le clic JavaScript
                            driver.execute_script("arguments[0].checked = true;", checkbox)
                            time.sleep(0.5)
        
                            # Vérifier si cela a fonctionné
                            if not checkbox.is_selected():
                                # Essayer le clic classique
                                checkbox.click()
    
                        print("✅ Case des conditions générales cochée")
    
                    except Exception as e:
                        print(f"❌ Erreur case à cocher : {e}")

                    # Pause avant validation
                    print("⏳ Pause avant validation...")
                    human_pause(2, 3)

                    # 5. Cliquer sur le bouton Valider
                    try:
                        print("🔍 Recherche du bouton Valider...")
                        validate_btn = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_validateButton"))
                        )
    
                        # Scroll vers le bouton
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", validate_btn)
                        time.sleep(2)
    
                        # Vérifier que le bouton est bien visible et activé
                        if validate_btn.is_displayed() and validate_btn.is_enabled():
                            print("✅ Bouton trouvé et prêt")
        
                            # Essayer plusieurs méthodes de clic
                            try:
                                # Méthode 1: Clic JavaScript
                                driver.execute_script("arguments[0].click();", validate_btn)
                                print("✅ Bouton 'Valider' cliqué (JavaScript)")
            
                            except Exception as click_e:
                                try:
                                    # Méthode 2: Clic classique
                                    validate_btn.click()
                                    print("✅ Bouton 'Valider' cliqué (classique)")
                
                                except Exception as click_e2:
                                    # Méthode 3: Submit du formulaire directement
                                    form = driver.find_element(By.ID, "ctl0_ctl2")
                                    form.submit()
                                    print("✅ Formulaire soumis directement")
                        else:
                            print("⚠️  Bouton trouvé mais pas interactif")
        
                        # Pause pour le traitement
                        print("⏳ Attente du traitement...")
                        human_pause(5, 8)
    
                    except Exception as e:
                        print(f"❌ Erreur bouton valider : {e}")
    
                        # Dernière tentative: soumission JavaScript
                        try:
                            print("🔄 Tentative de soumission JavaScript...")
                            driver.execute_script("document.getElementById('ctl0_ctl2').submit();")
                            print("✅ Formulaire soumis via JavaScript")
                            human_pause(5, 8)
                        except Exception as final_e:
                            print(f"❌ Échec final : {final_e}")

                    print("✅ Processus de remplissage du formulaire terminé")
                    
                    print("⬇️ Recherche du lien de téléchargement final...")
                    
                    # Attendre que la page se charge complètement après soumission du formulaire
                    try:
                        # Première vérification : attendre que le div contenant le lien soit présent
                        print("🔍 Vérification de la présence du conteneur de téléchargement...")
                        download_container = WebDriverWait(driver, 30).until(
                            EC.presence_of_element_located((By.ID, "ctl0_CONTENU_PAGE_panelEntrepriseDownloadDce"))
                        )
                        print("✅ Conteneur de téléchargement détecté")
                        
                        # Attendre un peu plus pour que le contenu se charge
                        human_pause(3, 5)
                        
                        # Méthodes multiples pour trouver le lien de téléchargement
                        final_download_link = None
                        link_text_final = ""
                        
                        # Méthode 1 : Par ID direct
                        try:
                            print("🔍 Méthode 1 : Recherche par ID direct...")
                            final_download_link = WebDriverWait(driver, 15).until(
                                EC.element_to_be_clickable((By.ID, "ctl0_CONTENU_PAGE_EntrepriseDowloadDce_nomDce"))
                            )
                            link_text_final = final_download_link.text.strip()
                            if link_text_final:
                                print(f"✅ Lien trouvé par ID : '{link_text_final}'")
                            else:
                                # Si le texte est vide, vérifier l'attribut href
                                href = final_download_link.get_attribute('href')
                                if href and href != "":
                                    print(f"✅ Lien trouvé par ID avec href : {href}")
                                else:
                                    final_download_link = None
                                    print("❌ Lien trouvé par ID mais sans contenu")
                        except Exception as e1:
                            print(f"❌ Méthode 1 échouée : {str(e1)[:100]}...")
                            final_download_link = None
                        
                        # Méthode 2 : Recherche dans le conteneur par tag 'a'
                        if final_download_link is None:
                            try:
                                print("🔍 Méthode 2 : Recherche de liens dans le conteneur...")
                                links_in_container = download_container.find_elements(By.TAG_NAME, "a")
                                for link in links_in_container:
                                    href = link.get_attribute('href')
                                    text = link.text.strip()
                                    if href and href != "" and href != driver.current_url:
                                        final_download_link = link
                                        link_text_final = text if text else "Télécharger"
                                        print(f"✅ Lien trouvé dans conteneur : '{link_text_final}' - {href}")
                                        break
                            except Exception as e2:
                                print(f"❌ Méthode 2 échouée : {str(e2)[:100]}...")
                        
                        # Méthode 3 : Recherche par XPath plus générique
                        if final_download_link is None:
                            try:
                                print("🔍 Méthode 3 : Recherche par XPath...")
                                xpath_selectors = [
                                    "//div[@id='ctl0_CONTENU_PAGE_panelEntrepriseDownloadDce']//a[@href and @href!='']",
                                    "//a[contains(@id, 'nomDce')]",
                                    "//a[contains(@href, 'download') or contains(@href, 'telecharger') or contains(@href, 'dce')]"
                                ]
                                
                                for xpath in xpath_selectors:
                                    try:
                                        elements = driver.find_elements(By.XPATH, xpath)
                                        for element in elements:
                                            href = element.get_attribute('href')
                                            if href and href != "" and href != driver.current_url:
                                                final_download_link = element
                                                link_text_final = element.text.strip() if element.text.strip() else "Télécharger"
                                                print(f"✅ Lien trouvé par XPath : '{link_text_final}' - {href}")
                                                break
                                        if final_download_link:
                                            break
                                    except Exception as xpath_e:
                                        continue
                            except Exception as e3:
                                print(f"❌ Méthode 3 échouée : {str(e3)[:100]}...")
                        
                        # Si aucun lien trouvé, sauvegarder la page pour debug
                        if final_download_link is None:
                            print("❌ Aucun lien de téléchargement trouvé")
                            # Afficher le contenu du conteneur pour debug
                            try:
                                container_html = download_container.get_attribute('innerHTML')
                                print(f"🔍 Contenu du conteneur de téléchargement :")
                                print(container_html[:500] + "..." if len(container_html) > 500 else container_html)
                            except:
                                pass
                            
                            # Sauvegarder la page complète
                            debug_filename_no_link = f"debug_no_download_link_{consultations_traitees}.html"
                            with open(os.path.join(download_dir, debug_filename_no_link), 'w', encoding='utf-8') as f_debug:
                                f_debug.write(driver.page_source)
                            print(f"🔍 Page sauvegardée pour debug : {debug_filename_no_link}")
                            
                            raise Exception("Aucun lien de téléchargement valide trouvé")
                        
                        # Effectuer le téléchargement
                        print(f"🔗 Lien de téléchargement final trouvé : '{link_text_final}'")
                        
                        # Scroll vers l'élément
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", final_download_link)
                        human_pause(1, 2)
                        
                        # Vérifier si l'élément est toujours cliquable
                        try:
                            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(final_download_link))
                        except:
                            print("⚠️ Élément peut-être pas complètement cliquable, tentative quand même...")
                        
                        # Tentative de clic (méthodes multiples)
                        click_success = False
                        
                        # Méthode 1 : Clic normal
                        try:
                            final_download_link.click()
                            click_success = True
                            print("✅ Clic normal réussi")
                        except Exception as click_e1:
                            print(f"❌ Clic normal échoué : {str(click_e1)[:100]}...")
                        
                        # Méthode 2 : Clic JavaScript
                        if not click_success:
                            try:
                                driver.execute_script("arguments[0].click();", final_download_link)
                                click_success = True
                                print("✅ Clic JavaScript réussi")
                            except Exception as click_e2:
                                print(f"❌ Clic JavaScript échoué : {str(click_e2)[:100]}...")
                        
                        # Méthode 3 : Navigation directe vers l'URL
                        if not click_success:
                            try:
                                href = final_download_link.get_attribute('href')
                                if href and href != "":
                                    driver.get(href)
                                    click_success = True
                                    print("✅ Navigation directe réussie")
                            except Exception as click_e3:
                                print(f"❌ Navigation directe échouée : {str(click_e3)[:100]}...")
                        
                        if not click_success:
                            raise Exception("Impossible de cliquer sur le lien de téléchargement")
                        
                        status = "Téléchargé (formulaire)"
                        consultations_telechargees += 1
                        print("✅✅ Dossier téléchargé avec succès après formulaire.")
                        human_pause(6, 10)  # Pause plus longue pour le téléchargement
                    
                    except Exception as form_download_e:
                        status = "Erreur formulaire/téléchargement"
                        error_msg = str(form_download_e)
                        log_error(f"Erreur processus formulaire/téléchargement pour {consultation_url} : {form_download_e}")
                        debug_filename_form = f"debug_page_form_dl_error_{consultations_traitees}.html"
                        with open(os.path.join(download_dir, debug_filename_form), 'w', encoding='utf-8') as f_debug:
                            f_debug.write(driver.page_source)
                        print(f"🔍 Page (erreur formulaire/DL) sauvegardée : {debug_filename_form}")
                        
                        # Informations supplémentaires pour le debug
                        try:
                            print("🔍 Informations de debug supplémentaires :")
                            print(f"URL actuelle : {driver.current_url}")
                            
                            # Vérifier si on est sur la bonne page
                            page_title = driver.title
                            print(f"Titre de la page : {page_title}")
                            
                            # Chercher tous les liens sur la page
                            all_links = driver.find_elements(By.TAG_NAME, "a")
                            download_related_links = []
                            for link in all_links:
                                href = link.get_attribute('href')
                                text = link.text.strip()
                                if href and any(keyword in href.lower() for keyword in ['download', 'telecharger', 'dce', '.pdf', '.zip', '.doc']):
                                    download_related_links.append(f"Texte: '{text}' - Href: {href}")
                            
                            if download_related_links:
                                print("📎 Liens potentiels de téléchargement trouvés :")
                                for link_info in download_related_links[:5]:  # Afficher les 5 premiers
                                    print(f"  - {link_info}")
                            else:
                                print("❌ Aucun lien de téléchargement potentiel trouvé")
                                
                        except Exception as debug_e:
                            print(f"Erreur lors du debug : {debug_e}")
                    
                    # --- FIN DU NOUVEAU PROCESSUS ---

                except Exception as download_error:
                    status = "Erreur téléchargement"
                    error_msg = str(download_error)
                    log_error(f"Erreur lors du téléchargement pour {consultation_url} : {download_error}")
                    print(f"❌ Erreur téléchargement : {download_error}")

                # Enregistrer l'activité dans le log
                log_activity(consultation_url, titre_consultation, "", status, error_msg)
                human_pause(2, 4)
                
            except Exception as e_outer: # Erreur dans le traitement d'une consultation (avant le bloc de téléchargement)
                status = "Erreur traitement consultation"
                error_msg = str(e_outer)
                log_error(f"Erreur traitement consultation {current_consultation_url_for_log} (ligne {i}) : {e_outer}")
                log_activity(current_consultation_url_for_log, titre_consultation, "", status, error_msg)
                # Sauvegarder la page en cas d'erreur ici aussi
                debug_filename_outer = f"debug_page_consult_error_{consultations_traitees}.html"
                try:
                    with open(os.path.join(download_dir, debug_filename_outer), 'w', encoding='utf-8') as f_debug:
                        f_debug.write(driver.page_source)
                    print(f"🔍 Page (erreur consultation) sauvegardée : {debug_filename_outer}")
                except Exception as e_save:
                    print(f"Impossible de sauvegarder la page de debug : {e_save}")
                continue # Passer à la consultation suivante
        
        print("\n" + "=" * 60)
        print("✅ SCRAPING TERMINÉ")
        print(f"📊 Consultations traitées : {consultations_traitees}")
        print(f"📥 Téléchargements réussis (via formulaire) : {consultations_telechargees}")
        print(f"📋 Log détaillé : {log_filename}")
        print(f"🚨 Log d'erreurs : {error_log_filename}")
        print(f"📂 Fichiers téléchargés dans : {download_dir}")
        
        rapport = create_download_report()
        if rapport:
            print(f"📄 Rapport détaillé : {rapport}")
        print("=" * 60)

except Exception as e_main_loop:
    log_error(f"Erreur majeure dans la boucle principale de traitement des offres : {e_main_loop}")
    print(f"❌ Erreur critique lors du traitement des consultations : {e_main_loop}")

print("\n🔍 Script terminé.")
input("Appuyez sur Entrée pour fermer le navigateur...")
driver.quit()


🚀 Démarrage du scraper de marchés publics
🔍 Recherche du site via Google...
🔗 Lien trouvé: https://www.marchespublics.gov.ma/
✅ Accès au site marchespublics.gov.ma réussi
Pas de pop-up/cookies géré sur le site des marchés publics ou déjà accepté.
📋 Accès aux consultations en cours...
✅ Page consultations en cours chargée
🏷️ Sélection de la catégorie Services...
✅ Catégorie 'Services' sélectionnée.
📅 Configuration des dates...
ERREUR: Erreur dans les champs de date : Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ctl0_CONTENU_PAGE_AdvancedSearch_dateRemisePlisStart"]"}
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff66890e415+77285]
	GetHandleVerifier [0x0x7ff66890e470+77376]
	(No symbol) [0x0x7ff6686d9a6a]
	(No symbol) [0x0x7ff668730406]
	(No symbol) [0x0x7ff6687306

KeyboardInterrupt: 